In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io

In [21]:
import torch
import torch.nn as nn
import albumentations as A
import numpy as np
import time
import os

"""
ResNet-based Keypoint Estimator
"""
class Configuration:
    def __init__(self):
        """
        self.temp = {
            'train_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/train_3_2_22_fem.csv',
            'val_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/val_3_2_22_fem.csv',
            'test_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/test_3_2_22_fem.csv'
        }
        """
        self.init = {
            'PROJECT_NAME': 'Keypoint Estimation',
            'MODEL_NAME': 'Tib_64KP',
            'RUN_NAME': time.strftime('%Y-%m-%d-%H-%M-%S'),
            'WANDB_RUN_GROUP': 'Local',
            'FAST_DEV_RUN': False,  # Runs inputted batches (True->1) and disables logging and some callbacks
            'MAX_EPOCHS': 10,
            'MAX_STEPS': -1,    # -1 means it will do all steps and be limited by epochs
            'STRATEGY': None    # This is the training strategy. Should be 'ddp' for multi-GPU (like HPG)
        }
        self.etl = {
            'RAW_DATA_FILE': -1,
            'DATA_DIR': "data",
            # Lol what is this?
            'KEYPOINT_DIRECTORY': "keypoints",
            'KEYPOINT_TXT_FILES': ['tib_KPlabels_16.txt'],
            'VAL_SIZE':  0.2,       # looks sus
            'TEST_SIZE': 0.01,      # I'm not sure these two mean what we think
            #'random_state': np.random.randint(1,50)
            # HHG2TG lol; deterministic to aid reproducibility
            'RANDOM_STATE': 42,

            'CUSTOM_TEST_SET': False,
            'TEST_SET_NAME': '/my/test/set.csv'
        }

        self.dataset = {
            'DATA_NAME': 'Ten_Dogs_64KP',
            'SUBSET_PIXELS': True,
            'IMAGE_HEIGHT': 1024,
            'IMAGE_WIDTH': 1024,
            'MODEL_TYPE': 'tib',        # how should we do this? not clear this is still best...
            'CLASS_LABELS': {0: 'bone', 1: 'background'},
            'NUM_KEY_POINTS': 64,
            'IMG_CHANNELS': 1,      # Is this different from self.module['NUM_IMAGE_CHANNELS']
            'STORE_DATA_RAM': False,
            'IMAGE_THRESHOLD': 0,
            'USE_ALBUMENTATIONS': False,

            # What do these do?
            'NUM_PRINT_IMG' : 1,
            'KP_PLOT_RAD' : 3,

            #'NUM_POINTS' : 128,

            'GAUSSIAN_STDDEV' : 5,
            'GAUSSIAN_AMP' : 1e3,

            'STORE_DATA_RAM' : False,

            'CROP_IMAGES' : False,
            'CROP_MIN_X' : 0.29,
            'CROP_MAX_X' : 0.84,
            'CROP_MIN_Y' : 0.45,
            'CROP_MAX_Y' : 0.95,
            
            'IMAGES_PER_GRID': 1,
            'per_grid_image_count_height' : 1, 
            'per_grid_image_count_width' : 1
        }

        """
        # segmentation_net_module needs to be below dataset because it uses dataset['IMG_CHANNELS']
        self.keypoint_net_module = {
            'NUM_KEY_POINTS': 128,
            'NUM_IMG_CHANNELS': self.dataset['IMG_CHANNELS']
        }
        """

        self.datamodule = {
            'IMAGE_DIRECTORY': '/media/sasank/LinuxStorage/Dropbox (UFL)/Canine Kinematics Data/TPLO_Ten_Dogs_grids/',
            'CKPT_FILE': None,
            'USE_NAIVE_TEST_SET': False,
            'BATCH_SIZE': 2,
            'SHUFFLE': True,        # Only for training; for test and val this is set in the datamodule script to False
            'NUM_WORKERS': 2,
            'PIN_MEMORY': False
            #'SUBSET_PIXELS': True - this is now in dataset
        }


        # hyperparameters for training
        self.hparams = {
            'LOAD_FROM_CHECKPOINT': False,
            'learning_rate': 1e-3
        }

        #self.transform = None
        self.transform = \
        A.Compose([
            # Let's do only rigid transformations for now
            A.HorizontalFlip(p=0.9),
            A.VerticalFlip(p=0.9),
            A.RandomRotate90(p=0.9),
            A.Transpose(p=0.9),
            #A.RandomGamma(always_apply=False, p = 0.5,gamma_limit=(10,300)),
            #A.ShiftScaleRotate(always_apply = False, p = 0.5,shift_limit=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-180,180), interpolation=0, border_mode=0, value=(0, 0, 0)),
            #A.Blur(always_apply=False, blur_limit=(3, 10), p=0.2),
            #A.Flip(always_apply=False, p=0.5),
            #A.InvertImg(always_apply=False, p=0.5),
            #A.MultiplicativeNoise(always_apply=False, p=0.25, multiplier=(0.1, 2), per_channel=True, elementwise=True)
            #A.ElasticTransform(always_apply=False, p=0.85, alpha=0.5, sigma=150, alpha_affine=50.0, interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, approximate=False),
            #A.CoarseDropout(always_apply = False, p = 0.25, min_holes = 1, max_holes = 100, min_height = 25, max_height=25),
        ],
        p=0.85)

In [25]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from skimage import io
import cv2

import pytorch_lightning as pl
import wandb

from loss import kp_loss



class KeypointDataset(torch.utils.data.Dataset):

    def __init__(self, config, evaluation_type, transform=None):
        """
        Args:
            config (config): Dictionary of vital constants about data.
            store_data_ram (boolean): Taken from config.
            evaluation_type (string): Dataset evaluation type (must be 'training', 'validation', or 'test')
            num_points (int): Taken from config.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        # Create local copies of the arguments
        self.config = config
        self.num_points = self.config.dataset['NUM_KEY_POINTS']
        self.transform = self.config.transform
        
        # Check that evaluation_type is valid and then store
        if evaluation_type in ['train', 'val', 'test', 'naive']:
            self.evaluation_type = evaluation_type
        else:
            raise Exception('Incorrect evaluation type! Must be either \'train\', \'val\', \'test\', or \'naive\'.')

        # Load the data from the big_data CSV file into a pandas dataframe
        #self.data = pd.read_csv(os.path.join(self.config.etl['DATA_DIR'], self.config.dataset['DATA_NAME'], self.evaluation_type + '_' + self.config.dataset['DATA_NAME'] + '.csv'))
        self.data = pd.read_csv('/home/sasank/Documents/GitRepos/Stifle-Keypoints/data/proc_64KP_data.csv')

    def __len__(self):
        return len(self.data) - 1   # Subtract 1 because the first row is the column names
    
    def __getitem__(self, idx):
        idx += 1    # Add 1 because the first row is the column names

        # Get the row of the dataframe
        row = self.data.iloc[idx]

        # Get the image name
        image_name = row['Image address']

        # Get the image
        image = io.imread(os.path.join(self.config.datamodule['IMAGE_DIRECTORY'], image_name))
        full_image = image             # Save full image (no subset_pixels) for visualization

        # Get the keypoint labels and segmentation labels
        if self.config.dataset['MODEL_TYPE'] == 'fem':
            kp_label = row['Femur 2D KP points']
            seg_label = io.imread(os.path.join(self.config.datamodule['IMAGE_DIRECTORY'], row['Fem label address']))
        elif self.config.dataset['MODEL_TYPE'] == 'tib':
            kp_label = row['Tibia 2D KP points']
            seg_label = io.imread(os.path.join(self.config.datamodule['IMAGE_DIRECTORY'], row['Tib label address']))
        else:
            raise Exception('Incorrect model type! Must be either \'fem\' or \'tib\'.')

        kp_label = kp_label[2:-2]
        kp_label = kp_label.split(']\n [')
        kp_label = [np.array([float(x) for x in list(filter(None, kp.split(' ')))]) for kp in kp_label]
        kp_label = np.array(kp_label)
        
        # * Subset Pixels
        if self.config.dataset['SUBSET_PIXELS'] == True:
            label_dst = np.zeros_like(seg_label)
            label_normed = cv2.normalize(seg_label, label_dst, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX)
            seg_label = label_normed

            kernel = np.ones((30,30), np.uint8)
            label_dilated = cv2.dilate(seg_label, kernel, iterations = 5)
            image_subsetted = cv2.multiply(label_dilated, image)
            image = image_subsetted

        image = torch.FloatTensor(image[None, :, :]) # Store as byte (to save space) then convert when called in __getitem__
        full_image = torch.FloatTensor(full_image[None, :, :]) # Store as byte (to save space) then convert when called in __getitem__
        seg_label = torch.FloatTensor(seg_label[None, :, :])
        #kp_label = torch.FloatTensor(kp_label.reshape(-1))      # Reshape to 1D array so that it's 2*num_keypoints long
        kp_label = torch.FloatTensor(kp_label)          # kp_label is of shape (num_keypoints, 2)
        assert kp_label.shape == (self.num_points, 2), "Keypoint label shape is incorrect!"
        #print("kp_label.shape:")
        #print(kp_label.shape)


    
        # Form sample and transform if necessary
        sample = {'image': image,
                    'img_name': image_name,
                    'kp_label': kp_label,
                    'seg_label': seg_label,
                    'full_image': full_image}
        assert self.transform is not None, "Transforms not implemented yet!"
        if self.transform and self.config.dataset['USE_ALBUMENTATIONS'] == True:
            sample = self.transform(sample)     # TODO: Implement transforms. Seems like we'll have to reshape the keypoints to be num_keypoints x 2 instead of 2*num_keypoints x 1
        return sample

In [26]:
config = Configuration()
dataset = KeypointDataset(config, 'train')

In [27]:
PRINT_DIR = '/media/sasank/LinuxStorage/Dropbox (UFL)/Canine Kinematics Data/image_check/pruned_tib/'
# Iterate through the dataset and print the shapes of the images and labels
for i in range(len(dataset)):
    print("image number: " + str(i))
    batch = dataset[i]
    images = batch['image']
    kp_labels = batch['kp_label']
    img_names = batch['img_name']
    title = ''




    num_images = images.shape[0]
    num_keypoints = kp_labels.shape[0]
    assert num_keypoints == 64, "Number of keypoints is incorrect!"
    images = images.cpu()
    kp_labels = kp_labels.cpu()
    kp_labels = kp_labels.numpy()

    output_image_vector = []

    for i in range(0, num_images):
        fig, ax = plt.subplots(1, 1, figsize=(10, 10), squeeze=False)

        # ! TODO: Is this the right way to do this? Is there something wrong here with offsets or something?
        #kp_labels[i][:, 0] = +1 * kp_labels[i][:, 0] * 1024
        kp_labels[:, 0] = +1 * kp_labels[:, 0] * 1024
        #kp_labels[i][:, 1] = -1 * kp_labels[i][:, 1] * 1024 + 1024
        kp_labels[:, 1] = -1 * kp_labels[:, 1] * 1024 + 1024
        # Do some stuff so that img is shown correctly
        img = images.numpy()
        img = np.transpose(img, (1, 2, 0))  # Transpose the output so that it's the same way as img
        img = np.dstack((img, img, img))    # Make it 3 channels
        ax[0][0].imshow((img * 255).astype(np.uint8))  # The multiplying by 255 and stuff is so it doesn't get clipped or something

        for j in range(num_keypoints):
            #ax[0][0].text(labels[i][j, 0], labels[i][j, 1], str(j), color='m')        # Silenced this for now since we have 64 keypoints
            ax[0][0].plot(kp_labels[j, 0], kp_labels[j, 1], color='orange', marker='.', markersize=5)
            #ax[0][0].plot([kp_labels[i][j, 0], kp_preds[i][j, 0]], [labels[i][j, 1], preds[i][j, 1]], color='limegreen', linestyle='-')
        image_name = img_names.split('/')[-1]    # Format img_names[i] so that only the part after the last '/' is shown
        ax[0][0].set_title(title + ' {}'.format(image_name))
        fig.savefig(os.path.join(PRINT_DIR, image_name))
        output_image_vector.append(fig)
        plt.close()
    

image number: 0
image number: 1
image number: 2
image number: 3
image number: 4
image number: 5
image number: 6
image number: 7
image number: 8
image number: 9
image number: 10
image number: 11
image number: 12
image number: 13
image number: 14
image number: 15
image number: 16
image number: 17
image number: 18
image number: 19
image number: 20
image number: 21
image number: 22
image number: 23
image number: 24
image number: 25
image number: 26
image number: 27
image number: 28
image number: 29
image number: 30
image number: 31
image number: 32
image number: 33
image number: 34
image number: 35
image number: 36
image number: 37
image number: 38
image number: 39
image number: 40
image number: 41
image number: 42
image number: 43
image number: 44
image number: 45
image number: 46
image number: 47
image number: 48
image number: 49
image number: 50
image number: 51
image number: 52
image number: 53
image number: 54
image number: 55
image number: 56
image number: 57
image number: 58
image n